In [ ]:
# יש להריץ ולהמתין מספר שניות עד שתופיע ספרה בתוך הסוגריים המרובעים
%pylab inline
from IPython.display import HTML
from matplotlib import animation

In [ ]:
# מותר לשנות את מה שכתוב בתא הזה בלבד

# נתוני התווך
chain_length = 100; # m אורך השרשרת
chain_spacing = 0.1; # m מרחק בין הגופים
m1 = 1; # kg מסת מחצית הגופים הראשונים
m2 = 1; # kg מסת מחצית הגופים השניים
k = 20; # N/m קבוע הקפיץ
end_type_left = 'close'; # {'close','open'} סוג הקצה בצד שמאל
end_type_right = 'close'; # {'close','open'} סוג הקצה בצד ימין

# נתוני המקור
source_location = 0; # m מיקום המקור
source_Amplitude = 1; # m אמפליטודת המקור
source_T = 447.2/6.5; # s זמן המחזור של המקור ההרמוני

# מיקום הגלאים
detector1 = 0; # m מיקום גלאי 1
detector2 = 50; # m מיקום גלאי 2

# פרמטרים נוספים לסימולציה
running_time = 500; # s זמן הרצת הסימולציה
dt_plot = running_time/50; # s כל כמה זמן לשרטט את הגרף
increase_y_axes_by = 5; # פי כמה להגדיל את גבולות הגרף בציר האנכי, ביחס לאמפליטודה
show_detectors = True; # האם להראות את גרפי הגלאים

In [ ]:
%%capture

# אין לשנות כלום בתא הזה - רק להריץ
# פרמטרים נוספים
dt = 0.01; # s צעד הזמן
marker_size = 1; # גודל הנקודות בגרפים

# חישובי ביניים
N_mass = int(ceil(chain_length/chain_spacing))+1; # מספר המסות בשרשרת
chain_length = (N_mass-1)*chain_spacing; # m חישוב מחדש של אורך השרשרת, מתוקן כך שהמסות יתאימו בדיוק
m_change_index = round(N_mass/2); # האינדקס של הגוף בו המסה משתנה
dt_plot_index = round(dt_plot/dt); # כל כמה לופים יש לשרטט את הגרף
N_frames = int(ceil(running_time/dt_plot))+1; # מספר שרטוטי הגרפים
source_index = round(source_location/chain_spacing); # האינדקס של המקור
detector1_index = round(detector1/chain_spacing); # האינדקס של הגלאי הראשון
detector2_index = round(detector2/chain_spacing); # האינדקס של הגלאי השני
source_location = source_index*chain_spacing; # m המיקום המתוקן של המקור
detector1 = detector1_index*chain_spacing; # m המיקום המתוקן של הגלאי הראשון
detector2 = detector2_index*chain_spacing; # m המיקום המתוקן של הגלאי השני 
if source_location == 0:
    end_type_left = 'source';
elif source_location == chain_length:
    end_type_right = 'source';    
    
# איתחול המערכים
x = arange(0,chain_length+chain_spacing,chain_spacing,float_); # מערך של קואורדינטות איקס
y = x*0; # מערך של הסטייה מנקודת שיווי-משקל
m = x*0; # מערך של מסת הגופים
m[0:m_change_index] = m1; # מסת מחצית הגופים הראשונים
m[m_change_index:] = m2; # מסת מחצית הגופים השניים
v = x*0; # מערך של מהירויות הגופים
a = x*0; # מערך של תאוצות הגופים
Nframes=int(ceil(running_time/dt_plot));
t_array = arange(0,(Nframes+1)*dt_plot,dt); # מערך של הזמנים
signal_detector1 = t_array*nan; # מערך של הגלאי הראשון
signal_detector2 = t_array*nan; # מערך של הגלאי השני

# איתחול הגרפים
fig=figure(figsize=(15, 7))
#fig = figure(1); # יצירת חלון לגרפים
clf(); # ניקוי החלון
ax_detector1 = fig.add_subplot(2,2,1); # יצירת גרף הגלאי הראשון
ax_detector2 = fig.add_subplot(2,2,3); # יצירת גרף הגלאי השני
ax_frame = fig.add_subplot(2,2,(2,4)); # יצירת גרף התמונות
# גרף התמונות
ax_frame.set_xlim(0,chain_length);
ax_frame.set_ylim(source_Amplitude*increase_y_axes_by*array([-1,1]));
ax_frame.set_xlabel('Position on chain [m]');
ax_frame.set_ylabel('Deviation from equilibrium [m]');
ax_frame.grid('on');
ax_frame.plot(detector1*ones(2),ylim(),'r',linewidth=3);
ax_frame.plot(detector2*ones(2),ylim(),'c',linewidth=3);
color1 = 'b.';
if m1==m2:
    color2 = color1;
else:
    color2 = 'g.';
line_chain1, = ax_frame.plot(x[0:m_change_index],y[0:m_change_index],color1,markersize=marker_size);
line_chain2, = ax_frame.plot(x[m_change_index:],y[m_change_index:],color2,markersize=marker_size);
ax_frame.title.set_text('0 s');
# גרפי הגלאים
ax_detector1.set_xlim(0,running_time);
ax_detector2.set_xlim(0,running_time);
ax_detector1.set_ylim(source_Amplitude*increase_y_axes_by*array([-1,1]));
ax_detector2.set_ylim(source_Amplitude*increase_y_axes_by*array([-1,1]));
ax_detector2.set_xlabel('Time [s]');
ax_detector1.set_ylabel('Deviation Detector 1 [m]');
ax_detector2.set_ylabel('Deviation Detector 2 [m]');
ax_detector1.title.set_text('Detectors');
ax_detector1.grid('on')
ax_detector2.grid('on')
line_detector1, = ax_detector1.plot(t_array,signal_detector1,'r.',markersize=marker_size);
line_detector2, = ax_detector2.plot(t_array,signal_detector2,'c.',markersize=marker_size);
if not show_detectors:
    line_detector1.set_visible(False)
    line_detector2.set_visible(False)


#for i in arange(0,int(ceil(running_time/dt))+1):
global t,a,m,y,v,dt,source_Amplitude,source_T,end_type_left,end_type_right
global line_detector1,line_detector2,detector1_index,detector2_index,ax_frame,line_chain1,line_chain2

def init():
    global i_t
    i_t=0
    
def animate(i_frame):
    global i_t,t,a,m,y,v,dt,source_Amplitude,source_T,end_type_left,end_type_right
    global line_detector1,line_detector2,detector1_index,detector2_index,ax_frame,line_chain1,line_chain2
    t_plot=0;
    while t_plot<dt_plot:
        t=i_t*dt
        i_t=i_t+1
        t_plot=t_plot+dt
        y[1:-1] = y[1:-1] + v[1:-1]*dt; # חישוב הסטיות החדשות, בהנחת מהירות קבועה

        y[source_index] = source_Amplitude*sin(2*pi/source_T*t); # המקור

        # חישוב הקצוות
        if end_type_left == 'close':
            y[0] = 0;
        elif end_type_left == 'open':
            y[0] = y[1];
        if end_type_right == 'close':
            y[-1] = 0;
        elif end_type_right == 'open':
            y[-1] = y[-2];

        a[1:-1]=k*(y[2:]+y[0:-2]-2*y[1:-1])/m[1:-1]; # חישוב התאוצות
        v[1:-1]=v[1:-1]+a[1:-1]*dt; # חישוב המהירויות
        
        signal_detector1[i_t] = y[detector1_index]; # שמירת מצב גלאי 1
        signal_detector2[i_t] = y[detector2_index]; # שמירת מצב גלאי 2

        
    line_chain1.set_ydata(y[0:m_change_index]); # עדכון גרף תמונה למחצית המסות הראשונה
    line_chain2.set_ydata(y[m_change_index:]); # עדכון גרף תמונה למחצית המסות השנייה
    ax_frame.title.set_text('%.1f s' %t); # עדכון הכותרת
    if show_detectors:
        line_detector1.set_ydata(signal_detector1); # עדכון גרף גלאי 1
        line_detector2.set_ydata(signal_detector2); # עדכון גרף גלאי 2

    #pause(0.0001);

anim=animation.FuncAnimation(fig, animate, frames=range(Nframes), interval=100, init_func=init);

In [ ]:
HTML(anim.to_jshtml())